In [ ]:
# CRIA O AMBIENTE VIRTUAL

conda create -n PEFT_PromptTuning python=3.11
conda activate PEFT_PromptTuning

In [ ]:
# INSTALA AS DEPENDENCIAS

pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install tqdm ipykernel transformers==4.42.4 datasets==2.19.1 peft==0.10.0 trl==0.9.4
# pip installaccelerate==0.30.1 tokenizers==0.19.1 bitsandbytes==0.43.1

python -m ipykernel install --user --name=PEFT_PromptTuning --display-name="PEFT_PromptTuning"

In [11]:
import pandas as pd

df = pd.read_json("dataset_classificacao_juridica.jsonl", lines=True)
print(df['area'].value_counts())

area
Direito Penal               120
Direito Processual Civil    119
Direito Empresarial         114
Direito Constitucional      113
Direito Civil               113
Direito do Trabalho         113
Direito do Consumidor       112
Direito Administrativo      109
Direito Tributário          109
Name: count, dtype: int64


In [2]:
# IMPORTA AS BIBLIOTECAS

import torch
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit
import numpy as np

/home/ricardo/.conda/envs/PEFT_PromptTuning/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# CONFIGURAÇÕES

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_FILE = "dataset_classificacao_juridica.jsonl"
OUTPUT_DIR = "./modelo_juridico_prompt_tuning"

In [4]:
# CARREGAR DATASET
full_dataset = load_dataset('json', data_files={'train': DATASET_FILE}, split='train')

# Mapeia os rótulos de texto para IDs numéricos (essencial para o modelo)
labels = full_dataset.unique("area")
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
print(f"Mapeamento de rótulos: {label2id}")

# Transforma os rótulos em IDs
def add_labels(example):
    example['label'] = label2id[example['area']]
    return example

full_dataset = full_dataset.map(add_labels)

# Divide o dataset em treino, validação e teste
train_test_split = full_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
test_validation_split = train_test_split['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

final_datasets = DatasetDict({
    'train': train_test_split['train'],
    'validation': test_validation_split['train'],
    'test': test_validation_split['test']
})

print("\nDataset final dividido:")
print(final_datasets)

Mapeamento de rótulos: {'Direito Processual Civil': 0, 'Direito Penal': 1, 'Direito Tributário': 2, 'Direito Civil': 3, 'Direito Administrativo': 4, 'Direito Constitucional': 5, 'Direito do Trabalho': 6, 'Direito do Consumidor': 7, 'Direito Empresarial': 8}

Dataset final dividido:
DatasetDict({
    train: Dataset({
        features: ['texto', 'area', 'label'],
        num_rows: 817
    })
    validation: Dataset({
        features: ['texto', 'area', 'label'],
        num_rows: 102
    })
    test: Dataset({
        features: ['texto', 'area', 'label'],
        num_rows: 103
    })
})


In [5]:
# TOKENIZA O TEXTO

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
def tokenize_function(examples):
    return tokenizer(examples["texto"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = final_datasets.map(tokenize_function, batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# CARREGAR MODELO

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,  # Tarefa de Classificação de Sequência
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=50,  # Tamanho do prompt virtual que será treinado
    prompt_tuning_init_text="Classifique o seguinte texto jurídico na área do direito correspondente:",
    tokenizer_name_or_path=MODEL_NAME,
)

# Carrega o modelo base para CLASSIFICAÇÃO DE SEQUÊNCIA
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels), # Informa ao modelo quantas classes existem
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True,
    # device_map="auto" # Usar se tiver VRAM suficiente
)

model.config.pad_token_id = tokenizer.eos_token_id

# Adiciona o adaptador de Prompt Tuning ao modelo
model = get_peft_model(model, peft_config)

# Imprime os parâmetros treináveis (será um número pequeno)
model.print_trainable_parameters()

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


trainable params: 52,864 || all params: 494,093,696 || trainable%: 0.01069918528165152


In [9]:
# EXECUTA O TREINAMENTO

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=2e-4, # Prompt Tuning geralmente usa uma taxa de aprendizado maior que LoRA
    num_train_epochs=12,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=False,
    overwrite_output_dir=True,
    report_to="none",
    lr_scheduler_type='cosine',
    warmup_steps=100,
)

# Instanciar o Trainer padrão
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

trainer.train()

print(f"Treinamento concluído. Salvando o adaptador em: {OUTPUT_DIR}")
trainer.save_model()

Epoch,Training Loss,Validation Loss
1,No log,2.168771
2,No log,2.392521
3,No log,2.148149
4,No log,2.069149
5,1.728600,2.211992
6,1.728600,1.942071
7,1.728600,1.815562
8,1.728600,1.763541
9,1.728600,1.723557
10,1.130700,1.552305


Treinamento concluído. Salvando o adaptador em: ./modelo_juridico_prompt_tuning


In [10]:
# VERIFICAÇÃO E DEMONSTRAÇÃO

import torch
import torch.nn.functional as F
from peft import PeftModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer

base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)

model_com_prompt = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model_com_prompt.config.pad_token_id = tokenizer.eos_token_id

# Garante que o modelo esteja em modo de avaliação (desativa dropout, etc.)
model_com_prompt.eval()

# Move o modelo para a GPU, se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_com_prompt.to(device)


def classificar_texto(texto):
    # Tokeniza a entrada, convertendo para tensores PyTorch e movendo para a GPU
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding=True).to(device)

    # Faz a predição sem calcular gradientes (mais rápido e economiza memória)
    with torch.no_grad():
        outputs = model_com_prompt(**inputs)

    # A saída do modelo são 'logits' (valores brutos)
    logits = outputs.logits

    # Aplica a função softmax para converter logits em probabilidades
    probabilities = F.softmax(logits, dim=-1)

    # Pega o índice da classe com a maior probabilidade
    predicted_class_id = torch.argmax(probabilities, dim=-1).item()

    # Usa o mapeamento id2label para obter o nome da classe
    predicted_class_label = model_com_prompt.config.id2label[predicted_class_id]
    
    # Pega a probabilidade da classe prevista
    confidence_score = probabilities[0][predicted_class_id].item()

    return {"label": predicted_class_label, "score": confidence_score}


texto_exemplo = "O réu foi condenado por homicídio qualificado, com pena base acima do mínimo legal."
resultado = classificar_texto(texto_exemplo)

print("\n--- Teste de Inferência Manual ---")
print(f"Texto: '{texto_exemplo}'")
print(f"Resultado da Classificação: {resultado}")

texto_exemplo_2 = "A empresa foi autuada por não recolher o PIS e a COFINS sobre o faturamento."
resultado_2 = classificar_texto(texto_exemplo_2)
print(f"\nTexto: '{texto_exemplo_2}'")
print(f"Resultado da Classificação: {resultado_2}")

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-0.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



--- Teste de Inferência Manual ---
Texto: 'O réu foi condenado por homicídio qualificado, com pena base acima do mínimo legal.'
Resultado da Classificação: {'label': 'Direito Processual Civil', 'score': 0.9767060875892639}

Texto: 'A empresa foi autuada por não recolher o PIS e a COFINS sobre o faturamento.'
Resultado da Classificação: {'label': 'Direito Processual Civil', 'score': 0.8986489176750183}
